Análisis de Componentes Principales (PCA) en el conjunto de datos del Titanic

El análisis de componentes principales (PCA) es una técnica de reducción de dimensionalidad que se utiliza para transformar un conjunto de datos en un nuevo sistema de coordenadas, donde las variables originales se expresan en términos de componentes principales no correlacionados. Esto es especialmente útil cuando se trata de conjuntos de datos con muchas características (dimensiones).

Paso 1: Preprocesamiento de datos

Carga de datos: Carga el conjunto de datos del Titanic en un DataFrame utilizando una biblioteca como pandas en Python.

In [1]:
import pandas as pd

# Carga el conjunto de datos
data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

Limpieza de datos: Maneja los valores faltantes y atípicos en el conjunto de datos. Puedes imputar valores faltantes con la media o la mediana y eliminar valores atípicos utilizando técnicas como el método IQR.

In [2]:
# Imputa valores faltantes con la mediana
data['Age'].fillna(data['Age'].median(), inplace=True)

# Elimina valores atípicos utilizando el método IQR
Q1 = data['Fare'].quantile(0.25)
Q3 = data['Fare'].quantile(0.75)
IQR = Q3 - Q1
data = data[(data['Fare'] >= Q1 - 1.5 * IQR) & (data['Fare'] <= Q3 + 1.5 * IQR)]

Codificación de variables categóricas: Convierte las variables categóricas en variables numéricas utilizando la codificación one-hot.

In [3]:
# Codificación one-hot de variables categóricas
data_encoded = pd.get_dummies(data, columns=['Sex', 'Embarked'])

Normalización de características: Normaliza las características para que tengan media cero y desviación estándar uno.

In [4]:
from sklearn.preprocessing import StandardScaler

# Inicializa el escalador
scaler = StandardScaler()

# Ajusta el escalador a los datos y transforma las características
scaled_features = scaler.fit_transform(data_encoded[['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']])

Paso 2: Cálculo de PCA

Cálculo de la matriz de covarianza: Calcula la matriz de covarianza de las características normalizadas.

In [6]:
import numpy as np

# Calcula la matriz de covarianza
cov_matrix = np.cov(scaled_features, rowvar=False)

In [7]:
cov_matrix

array([[ 1.00129199,  0.08582635, -0.35005884, -0.33543273, -0.20444529],
       [ 0.08582635,  1.00129199, -0.5905382 ,  0.37086645,  0.33727968],
       [-0.35005884, -0.5905382 ,  1.00129199,  0.11409018,  0.08355915],
       [-0.33543273,  0.37086645,  0.11409018,  1.00129199,  0.41071228],
       [-0.20444529,  0.33727968,  0.08355915,  0.41071228,  1.00129199]])

Obtención de los autovalores y autovectores: Calcula los autovalores y autovectores de la matriz de covarianza.

In [8]:
# Calcula los autovalores y autovectores
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

Selección de componentes principales: Ordena los autovalores de mayor a menor y selecciona los primeros componentes principales que explican la mayor parte de la varianza acumulada.

In [9]:
# Ordena los autovalores en orden descendente
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalues = eigenvalues[sorted_indices]

# Calcula la varianza acumulada
total_variance = np.sum(sorted_eigenvalues)
explained_variance_ratio = sorted_eigenvalues / total_variance
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

Paso 3: Transformación de datos

Construcción de la matriz de proyección: Construye la matriz de proyección utilizando los autovectores seleccionados.

In [10]:
# Selecciona los primeros k autovectores (donde k es el número de componentes principales deseados)
num_components = 2  # Por ejemplo, selecciona los 2 primeros componentes principales
selected_eigenvectors = eigenvectors[:, sorted_indices[:num_components]]

# Construye la matriz de proyección
projection_matrix = selected_eigenvectors

Transformación de datos: Multiplica la matriz de datos preprocesados por la matriz de proyección para obtener los datos transformados en el espacio de componentes principales.

In [11]:
# Transforma los datos originales en el espacio de componentes principales
transformed_data = np.dot(scaled_features, projection_matrix)

Paso 4: Análisis e interpretación

Exploración de componentes principales: Examina los componentes principales y sus valores asociados para entender qué características originales contribuyen más a cada componente

In [13]:
# Imprime los autovectores (componentes principales) junto con la importancia de cada variable
for i, component in enumerate(selected_eigenvectors.T):
    print(f"Componente principal {i + 1}: {component}")
    print(f"Importancia de variables: {component * sorted_eigenvalues[i]}")

Componente principal 1: [-0.23880555  0.54747575 -0.15235618  0.57438715  0.53861787]
Importancia de variables: [-0.44157875  1.0123452  -0.28172399  1.06210746  0.99596597]
Componente principal 2: [-0.53298877 -0.43265652  0.67417223  0.22342326  0.15590111]
Importancia de variables: [-0.91362739 -0.74164197  1.15563825  0.38298295  0.26723926]


Varianza explicada: Calcula la varianza explicada por cada componente principal y la varianza acumulada.

In [14]:
# Imprime la varianza explicada por cada componente principal y la varianza acumulada
for i, explained_var in enumerate(explained_variance_ratio):
    print(f"Varianza explicada por el componente principal {i + 1}: {explained_var:.4f}")
    print(f"Varianza acumulada hasta el componente principal {i + 1}: {cumulative_variance_ratio[i]:.4f}")

Varianza explicada por el componente principal 1: 0.3693
Varianza acumulada hasta el componente principal 1: 0.3693
Varianza explicada por el componente principal 2: 0.3424
Varianza acumulada hasta el componente principal 2: 0.7117
Varianza explicada por el componente principal 3: 0.1359
Varianza acumulada hasta el componente principal 3: 0.8476
Varianza explicada por el componente principal 4: 0.1069
Varianza acumulada hasta el componente principal 4: 0.9546
Varianza explicada por el componente principal 5: 0.0454
Varianza acumulada hasta el componente principal 5: 1.0000
